<a href="https://colab.research.google.com/github/kronze1996/Product-Recommendation-Engine/blob/main/Kartikey_Sharma_Product_Recommendation_Engine_Capstone_Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Build a recommender engine that reviews customer ratings and purchase history to recommend items and improve sales. </u></b>

### Amazon.com is one of the largest electronic commerce and cloud computing companies.

### Just a few Amazon related facts:

### They lost $4.8 million in August 2013, when their website went down for 40 mins. They hold the patent on 1-Click buying, and licenses it to Apple. Their Phoenix fulfilment centre is a massive 1.2 million square feet. Amazon relies heavily on a Recommendation engine that reviews customer ratings and purchase history to recommend items and improve sales.


### This is a dataset related to over 2 Million customer reviews and ratings of Beauty related products sold on their website.

### It contains

* ### the unique UserId (Customer Identification),
* ### the product ASIN (Amazon's unique product identification code for each product),
* ### Ratings (ranging from 1-5 based on customer satisfaction) and
* ### the Timestamp of the rating (in UNIX time)

### This dataset contains product reviews and metadata from Amazon, including 142.8 million reviews spanning May 1996 - July 2014.

### This dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features), and links (also viewed/also bought graphs).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame 

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt



import re
import string
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics import mean_squared_error

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
path = "/content/drive/MyDrive/AlmaBetter/Cohort Aravali/Module 4/Week 4/ratings_Beauty.csv"
df = pd.read_csv(path)
review_df = pd.read_json('/content/drive/MyDrive/AlmaBetter/Cohort Aravali/Module 4/Week 4/reviews_Beauty_5 (1).json.gz',lines = True)

#Exploratory Data Analysis

In [4]:
df.drop_duplicates(inplace=True)
df.shape

(2023070, 4)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2023070 entries, 0 to 2023069
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   UserId     object 
 1   ProductId  object 
 2   Rating     float64
 3   Timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 77.2+ MB


In [6]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198502 entries, 0 to 198501
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      198502 non-null  object
 1   asin            198502 non-null  object
 2   reviewerName    197116 non-null  object
 3   helpful         198502 non-null  object
 4   reviewText      198502 non-null  object
 5   overall         198502 non-null  int64 
 6   summary         198502 non-null  object
 7   unixReviewTime  198502 non-null  int64 
 8   reviewTime      198502 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.6+ MB


In [7]:
df.head(20)

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200
5,AKJHHD5VEH7VG,0762451459,5.0,1404518400
6,A1BG8QW55XHN6U,1304139212,5.0,1371945600
7,A22VW0P4VZHDE3,1304139220,5.0,1373068800
8,A3V3RE4132GKRO,130414089X,5.0,1401840000
9,A327B0I7CYTEJC,130414643X,4.0,1389052800


In [8]:
df['ProductId'].value_counts()

B001MA0QY2    7533
B0009V1YR8    2869
B0043OYFKU    2477
B0000YUXI0    2143
B003V265QW    2088
              ... 
B001JYYI72       1
B0048JT1P4       1
B007GE37U8       1
B00K212FZU       1
B00CCYT11G       1
Name: ProductId, Length: 249274, dtype: int64

In [9]:
review_df.head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
5,A3BTN14HIZET6Z,7806397051,"S. M. Randall ""WildHorseWoman""","[1, 2]","I was very happy to get this palette, now I wi...",5,Very nice palette!,1365984000,"04 15, 2013"
6,A1Z59RFKN0M5QL,7806397051,"tasha ""luvely12b""","[1, 3]",PLEASE DONT DO IT! this just rachett the palet...,1,smh!!!,1376611200,"08 16, 2013"
7,AWUO9P6PL1SY8,7806397051,TreMagnifique,"[0, 1]","Chalky,Not Pigmented,Wears off easily,Not a Co...",2,"Chalky, Not Pigmented, Wears off easily, Not a...",1378252800,"09 4, 2013"
8,A3LMILRM9OC3SA,9759091062,NaN,"[0, 0]",Did nothing for me. Stings when I put it on. I...,2,"no Lightening, no Brightening,......NOTHING",1405209600,"07 13, 2014"
9,A30IP88QK3YUIO,9759091062,Amina Bint Ibraheem,"[0, 0]",I bought this product to get rid of the dark s...,3,Its alright,1388102400,"12 27, 2013"


Exploring the data sets

In [10]:
df[df['UserId']=='A1YJEY40YUW4SE']

,UserId,ProductId,Rating,Timestamp
340,A1YJEY40YUW4SE,7806397051,1.0,1391040000
440062,A1YJEY40YUW4SE,B000V6BCSW,2.0,1391040000
807482,A1YJEY40YUW4SE,B0020YLEYK,5.0,1328140800
942164,A1YJEY40YUW4SE,B002WLWX82,1.0,1391040000
1177542,A1YJEY40YUW4SE,B004756YJA,5.0,1318896000
1337973,A1YJEY40YUW4SE,B004ZT0SSG,5.0,1318896000
1538217,A1YJEY40YUW4SE,B006XL55FK,5.0,1391040000
1982381,A1YJEY40YUW4SE,B00GZTXTZI,4.0,1391040000


In [11]:
df['Rating'].value_counts()

5.0    1248721
4.0     307740
1.0     183784
3.0     169791
2.0     113034
Name: Rating, dtype: int64

In [12]:
df['UserId'].value_counts()

A3KEZLJ59C1JVH    389
A281NPSIMI1C2R    336
A3M174IC0VXOS2    326
A2V5R832QCSOMX    278
A3LJLRIZL38GG3    276
                 ... 
ALHIEOHZ5455        1
AC7W5TPN0Q24G       1
A1AMMRALBHMSCP      1
A3TZJTWZRNGRT9      1
A13VLLIC52VWIN      1
Name: UserId, Length: 1210271, dtype: int64

In [13]:
review_df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [14]:
review_df['asin']

0         7806397051
1         7806397051
2         7806397051
3         7806397051
4         7806397051
             ...    
198497    B00LLPT4HI
198498    B00LLPT4HI
198499    B00LLPT4HI
198500    B00LLPT4HI
198501    B00LLPT4HI
Name: asin, Length: 198502, dtype: object

#Content Based Filtering

## <u><b> Content-Based Filtering </b></u>

### This method uses only information about the description and attributes of the items users has previously consumed to model user's preferences. In other words, these algorithms try to recommend items that are similar to those that a user liked in the past (or is examining in the present). In particular, various candidate items are compared with items previously rated by the user and the best-matching items are recommended. 


Recommender systems have a problem known as user cold-start, in which it is hard to provide personalized recommendations for users with none or a very few number of consumed items, due to the lack of information to model their preferences.  

For this reason, we are keeping in the dataset only users with at least 5 interactions.

In [38]:
users_interactions_count_df = df.groupby(['UserId', 'ProductId']).size().groupby('UserId').size()
print('# of users: %d' % len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['UserId']]
print('# of users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# of users: 1210271
# of users with at least 5 interactions: 52374


In [39]:
users_with_enough_interactions_df.head()

,UserId
0,A00414041RD0BXM6WK0GX
1,A00473363TJ8YSZ3YAGG9
2,A00700212KB3K0MVESPIY
3,A00728291XHL0K3ETGQOG
4,A0078719IR14X3NNUG0F


Checking for user with 1 interaction

In [40]:
users_with_enough_interactions_df['UserId']

0        A00414041RD0BXM6WK0GX
1        A00473363TJ8YSZ3YAGG9
2        A00700212KB3K0MVESPIY
3        A00728291XHL0K3ETGQOG
4         A0078719IR14X3NNUG0F
                 ...          
52369            AZZTOUKVTUMVM
52370            AZZU6NXB8YJN9
52371            AZZV9PDNMCOZW
52372            AZZYW4YOE1B6E
52373            AZZZLM1E5JJ8C
Name: UserId, Length: 52374, dtype: object

In [41]:
print('# of interactions: %d' % len(df))
interactions_from_selected_users_df = df.merge(users_with_enough_interactions_df,how = 'right',left_on = 'UserId',right_on = 'UserId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 2023070
# of interactions from users with at least 5 interactions: 469771


In [42]:
users_with_enough_interactions_df.head(10)

,UserId
0,A00414041RD0BXM6WK0GX
1,A00473363TJ8YSZ3YAGG9
2,A00700212KB3K0MVESPIY
3,A00728291XHL0K3ETGQOG
4,A0078719IR14X3NNUG0F
5,A01049491OU9FM5JV4TEW
6,A01198201H0E3GHV2Z17I
7,A01247753D6GFZD87MUV8
8,A01288351ESHZ2KNAXBJ7
9,A0139874ED7NYUB55TSR


In [43]:
interactions_from_selected_users_df[interactions_from_selected_users_df['UserId']=='A27TKCMDYFCFOY']

,UserId,ProductId,Rating,Timestamp


In [44]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['UserId', 'ProductId'])['Rating'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 469771


,UserId,ProductId,Rating
0,A00414041RD0BXM6WK0GX,B007IY97U0,2.000000
1,A00414041RD0BXM6WK0GX,B00870XLDS,1.584963
2,A00414041RD0BXM6WK0GX,B008MIRO88,1.000000
3,A00414041RD0BXM6WK0GX,B00BQYYMN0,2.000000
4,A00414041RD0BXM6WK0GX,B00GRTQBTM,2.584963
5,A00414041RD0BXM6WK0GX,B00HFP4JZU,2.584963
6,A00414041RD0BXM6WK0GX,B00JM8Z52O,2.321928
7,A00473363TJ8YSZ3YAGG9,B000052YQU,1.584963
8,A00473363TJ8YSZ3YAGG9,B00016XA0K,2.000000
9,A00473363TJ8YSZ3YAGG9,B000FABN7E,2.584963


In [45]:
from sklearn.model_selection import train_test_split
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['UserId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 375816
# interactions on Test set: 93955


In [46]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('UserId')
interactions_train_indexed_df = interactions_train_df.set_index('UserId')
interactions_test_indexed_df = interactions_test_df.set_index('UserId')

In [ ]:
newDf = pd.merge(review_df, df, on=['asin'])

In [48]:
#Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_list = stopwords.words('english')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = review_df['asin'].tolist()
tfidf_matrix = vectorizer.fit_transform(review_df['reviewText'] + "" + review_df['summary'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<198502x2366 sparse matrix of type '<class 'numpy.float64'>'
	with 6780848 stored elements in Compressed Sparse Row format>

In [49]:
stopwords_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [50]:
tfidf_feature_names

['00',
 '10',
 '10 minutes',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '15 minutes',
 '16',
 '20',
 '20 minutes',
 '24',
 '25',
 '2nd',
 '30',
 '30 minutes',
 '34',
 '40',
 '45',
 '50',
 '60',
 '8217',
 '99',
 'able',
 'able use',
 'absolute',
 'absolutely',
 'absolutely love',
 'absorb',
 'absorbed',
 'absorbs',
 'absorbs quickly',
 'accidentally',
 'achieve',
 'acid',
 'acne',
 'acne prone',
 'across',
 'acrylic',
 'active',
 'actual',
 'actually',
 'add',
 'added',
 'adding',
 'addition',
 'additional',
 'adds',
 'admit',
 'adult',
 'advertised',
 'affordable',
 'afraid',
 'african',
 'african american',
 'afterwards',
 'age',
 'aging',
 'ago',
 'agree',
 'air',
 'air dry',
 'alcohol',
 'allergic',
 'allergies',
 'allow',
 'allows',
 'almond',
 'almost',
 'almost like',
 'aloe',
 'alone',
 'along',
 'alot',
 'already',
 'also',
 'also great',
 'also like',
 'also love',
 'also use',
 'also used',
 'alternative',
 'although',
 'always',
 'always use',
 'amazed',
 'amazing',
 'amazi

In [63]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['ProductId'])
    
    user_item_strengths = np.array(interactions_person_df['Rating']).reshape(-1,1)
    
    # Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_full_df[interactions_full_df['ProductId'].isin(review_df['asin'])].set_index('UserId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [65]:
interactions_indexed_df = interactions_full_df[interactions_full_df['ProductId'].isin(review_df['asin'])].set_index('UserId')
interactions_indexed_df.head()

,ProductId,Rating
UserId,,
A00414041RD0BXM6WK0GX,B007IY97U0,2.000000
A00414041RD0BXM6WK0GX,B00870XLDS,1.584963
A00414041RD0BXM6WK0GX,B008MIRO88,1.000000
A00414041RD0BXM6WK0GX,B00BQYYMN0,2.000000
A00414041RD0BXM6WK0GX,B00GRTQBTM,2.584963


In [64]:
user_profiles = build_users_profiles()
len(user_profiles)

ValueError: ignored

In [ ]:
user_profiles

In [ ]:
user_profile = user_profiles[-1479311724257856983]
print(user_profile.shape)

pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[-1479311724257856983].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

In [ ]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        
        # Compute the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        
        # Get the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        # Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(articles_df)

In [ ]:
def get_items_interacted(person_id, interactions_df):
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [ ]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:

    # Function for getting the set of items which a user has not interacted with
    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    # Function to verify whether a particular item_id was present in the set of top N recommended items
    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index
    
    # Function to evaluate the performance of model for each user
    def evaluate_model_for_user(self, model, person_id):
        
        # Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])
            
        interacted_items_count_testset = len(person_interacted_items_testset) 

        # Getting a ranked recommendation list from the model for a given user
        person_recs_df = model.recommend_items(person_id, items_to_ignore=get_items_interacted(person_id, interactions_train_indexed_df),topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        
        # For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            
            # Getting a random sample of 100 items the user has not interacted with
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, seed=item_id%(2**32))

            # Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            # Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            
            # Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        # Recall is the rate of the interacted items that are ranked among the Top-N recommended items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    
    # Function to evaluate the performance of model at overall level
    def evaluate_model(self, model):
        
        people_metrics = []
        
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):    
            person_metrics = self.evaluate_model_for_user(model,person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
            
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics).sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)


print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

#Collaborative Filtering

## <u><b> Collaborative Filtering </b></u> 

This method makes automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on a set of items, A is more likely to have B's opinion for a given item than that of a randomly chosen person.   



In [18]:
review_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [19]:
review_df.shape

(198502, 9)

In [20]:
count = review_df.groupby("asin", as_index=False).count()
count

,asin,reviewerID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,7806397051,8,8,8,8,8,8,8,8
1,9759091062,11,10,11,11,11,11,11,11
2,9788072216,5,5,5,5,5,5,5,5
3,9790790961,6,6,6,6,6,6,6,6
4,9790794231,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...
12096,B00L5KTZ0K,15,15,15,15,15,15,15,15
12097,B00L6Q3BH6,5,5,5,5,5,5,5,5
12098,B00LCEROA2,9,9,9,9,9,9,9,9
12099,B00LG63DOM,10,10,10,10,10,10,10,10


In [21]:
count.drop(['reviewerName'	,'helpful'	,'reviewText','unixReviewTime'	,'reviewTime'],axis=1,inplace=True)
count

,asin,reviewerID,overall,summary
0,7806397051,8,8,8
1,9759091062,11,11,11
2,9788072216,5,5,5
3,9790790961,6,6,6
4,9790794231,5,5,5
...,...,...,...,...
12096,B00L5KTZ0K,15,15,15
12097,B00L6Q3BH6,5,5,5
12098,B00LCEROA2,9,9,9
12099,B00LG63DOM,10,10,10


In [22]:
dfMerged = pd.merge(review_df, count, on=['asin'])
dfMerged

,reviewerID_x,asin,reviewerName,helpful,reviewText,overall_x,summary_x,unixReviewTime,reviewTime,reviewerID_y,overall_y,summary_y
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014",8,8,8
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014",8,8,8
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013",8,8,8
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013",8,8,8
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013",8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014",7,7,7
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014",7,7,7
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014",7,7,7
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014",7,7,7


In [23]:
dfMerged[dfMerged['reviewerID_y']<5]

,reviewerID_x,asin,reviewerName,helpful,reviewText,overall_x,summary_x,unixReviewTime,reviewTime,reviewerID_y,overall_y,summary_y


In [24]:
#rename column
dfMerged["totalReviewers"] = dfMerged["reviewerID_y"]
dfMerged["overallScore"] = dfMerged["overall_x"]
dfMerged["summaryReview"] = dfMerged["summary_x"]

dfNew = dfMerged[['asin','summaryReview','overallScore',"totalReviewers"]]

In [25]:
dfNew

,asin,summaryReview,overallScore,totalReviewers
0,7806397051,Don't waste your money,1,8
1,7806397051,OK Palette!,3,8
2,7806397051,great quality,4,8
3,7806397051,Do not work on my face,2,8
4,7806397051,It's okay.,3,8
...,...,...,...,...
198497,B00LLPT4HI,A little dab...,5,7
198498,B00LLPT4HI,Pure organic raw shea butter,5,7
198499,B00LLPT4HI,One Pound Organic Grade A Unrefined Shea Butter,5,7
198500,B00LLPT4HI,This stuff is amazing!,5,7


In [26]:
# Selecting products with more than 40 reviews
dfMerged = dfMerged.sort_values(by='totalReviewers', ascending=False)
dfCount = dfMerged[dfMerged.totalReviewers >= 40]
dfCount

,reviewerID_x,asin,reviewerName,helpful,reviewText,overall_x,summary_x,unixReviewTime,reviewTime,reviewerID_y,overall_y,summary_y,totalReviewers,overallScore,summaryReview
112506,A3BP5ZF51CHZOE,B004OHQR1Q,Caitlyn Johnson,"[7, 9]",first off.... i ordered these expecting there ...,1,crap!,1382572800,"10 24, 2013",431,431,431,431,1,crap!
112578,A3796JLADKK5Z7,B004OHQR1Q,Freckvanilla,"[0, 0]",I use this all the time helps out with my desi...,5,YOU NEED THIS!!,1372118400,"06 25, 2013",431,431,431,431,5,YOU NEED THIS!!
112580,A22NETHJ4KTWJP,B004OHQR1Q,gadgetGirl,"[0, 0]",Love the array of colors and the different siz...,5,Great product at a very affordable price,1367798400,"05 6, 2013",431,431,431,431,5,Great product at a very affordable price
112581,A1L9ZTGO75717E,B004OHQR1Q,gatorsgate,"[0, 0]",I would recommend these dotting tools to anyon...,5,Dotting 5 X 2 Way Marbleizing Dotting Pen Set,1398124800,"04 22, 2014",431,431,431,431,5,Dotting 5 X 2 Way Marbleizing Dotting Pen Set
112582,A3DDZQYUAE9WNP,B004OHQR1Q,gee,"[0, 0]",These are easy to use. Gets the job done and c...,5,Love the look.,1375833600,"08 7, 2013",431,431,431,431,5,Love the look.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106492,A7L44HUPRXYQ0,B004DHYZU2,Panda P,"[4, 6]",So I bought this after I got a free sample in ...,2,Ehh,1332892800,"03 28, 2012",40,40,40,40,2,Ehh
106491,A1SWKVROFA2FC8,B004DHYZU2,Onefortheroad,"[0, 0]",This stuff is amazing! It definitely is crack ...,5,Works great!,1384214400,"11 12, 2013",40,40,40,40,5,Works great!
106489,A39LRXD7THXB4B,B004DHYZU2,"M. Williams ""MBW""","[1, 2]",I love this product! I did mix it with another...,5,Excellent!,1375833600,"08 7, 2013",40,40,40,40,5,Excellent!
106490,A16QJ07X360VDK,B004DHYZU2,Nicky Y. Schleider,"[0, 0]",i've used this product several times. it's ok...,3,crack,1378252800,"09 4, 2013",40,40,40,40,3,crack


In [27]:
dfProductReview = review_df.groupby("asin", as_index=False).mean()
ProductReviewSummary = dfCount.groupby("asin")["summaryReview"].apply(list)
ProductReviewSummary = pd.DataFrame(ProductReviewSummary)
ProductReviewSummary.to_csv("ProductReviewSummary.csv")

In [28]:
dfProductReview

,asin,overall,unixReviewTime
0,7806397051,2.625000,1.382087e+09
1,9759091062,3.090909,1.390930e+09
2,9788072216,5.000000,1.342552e+09
3,9790790961,4.333333,1.378858e+09
4,9790794231,3.600000,1.298212e+09
...,...,...,...
12096,B00L5KTZ0K,5.000000,1.405014e+09
12097,B00L6Q3BH6,4.400000,1.400734e+09
12098,B00LCEROA2,5.000000,1.404643e+09
12099,B00LG63DOM,4.800000,1.405823e+09


In [29]:
dfProductReview.rename(columns={"asin":"asin"},inplace=True)

In [30]:
df3 = pd.read_csv("ProductReviewSummary.csv")
df3 = pd.merge(df3, dfProductReview, on="asin", how='inner')

In [31]:
df3 = df3[['asin','summaryReview','overall']]
df3

,asin,summaryReview,overall
0,B000050B6U,"['Okay', ""Best I've used so far"", 'The BEST!',...",4.313725
1,B000052WYD,"['Good Buy', 'Not a fan', 'Not impressed.', 'g...",3.680000
2,B000052YJM,"['Skin Has Not Improved', 'No more breakouts',...",4.292683
3,B000052YM4,['Cleans well but too strong for sensitive ski...,3.909091
4,B000052YM7,"['Fantastic Product', 'This lotion helps with ...",4.010753
...,...,...,...
988,B00KH6F6TM,"['Four Stars', 'High quality mini facial-skin/...",4.979167
989,B00KHGIK54,"['Preventative Measure!', 'Beautiful Results; ...",4.832061
990,B00KHH2VOY,"['Great Soap', 'LOVE this soap!!', 'another fa...",4.877049
991,B00KQBR9FM,"['Love at first use', 'very hydrating', 'Love ...",4.745098


In [32]:
df3.shape

(993, 3)

Text cleaning

In [33]:
regEx = re.compile('[^a-z]+')
def cleanReviews(reviewText):
    reviewText = reviewText.lower()
    reviewText = regEx.sub(' ', reviewText).strip()
    return reviewText

In [34]:
df3["summaryClean"] = df3["summaryReview"].apply(cleanReviews)
df3 = df3.drop_duplicates(['overall'], keep='last')
df3 = df3.reset_index()

Vectorization

In [35]:
reviews = df3["summaryClean"] 
countVector = CountVectorizer(max_features = 300, stop_words='english') 
transformedReviews = countVector.fit_transform(reviews) 

dfReviews = DataFrame(transformedReviews.A, columns=countVector.get_feature_names())
dfReviews = dfReviews.astype(int)

###Train Test split

In [36]:
# First let's create a dataset called X
X = np.array(dfReviews)
 # create train and test
tpercent = 0.9
tsize = int(np.floor(tpercent * len(dfReviews)))
dfReviews_train = X[:tsize]
dfReviews_test = X[tsize:]
#len of train and test
lentrain = len(dfReviews_train)
lentest = len(dfReviews_test)

##Model Based Approach

###KNN Recommendation

In [37]:
neighbor = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(dfReviews_train)

# Let's find the k-neighbors of each point in object X. To do that we call the kneighbors() function on object X.
distances, indices = neighbor.kneighbors(dfReviews_train)

In [38]:
for i in range(lentest):
    a = neighbor.kneighbors([dfReviews_test[i]])
    related_product_list = a[1]

    first_related_product = [item[0] for item in related_product_list]
    first_related_product = str(first_related_product).strip('[]')
    first_related_product = int(first_related_product)
    second_related_product = [item[1] for item in related_product_list]
    second_related_product = str(second_related_product).strip('[]')
    second_related_product = int(second_related_product)
    
    print ("Based on product reviews, for ", df3["asin"][lentrain + i] ," average rating is ",df3["overall"][lentrain + i])
    print ("The first similar product is ", df3["asin"][first_related_product] ," average rating is ",df3["overall"][first_related_product])
    print ("The second similar product is ", df3["asin"][second_related_product] ," average rating is ",df3["overall"][second_related_product])
    print ("-----------------------------------------------------------")

Based on product reviews, for  B00GTBZNP6  average rating is  4.263888888888889
The first similar product is  B00A0IWTBA  average rating is  4.197802197802198
The second similar product is  B004UKENRC  average rating is  4.23943661971831
-----------------------------------------------------------
Based on product reviews, for  B00GTBZOVE  average rating is  4.112676056338028
The first similar product is  B006L6A10G  average rating is  4.414285714285715
The second similar product is  B00GTBZ85Q  average rating is  4.178571428571429
-----------------------------------------------------------
Based on product reviews, for  B00GTBZWPW  average rating is  4.184615384615385
The first similar product is  B00GTBZHCA  average rating is  4.303571428571429
The second similar product is  B006L6A10G  average rating is  4.414285714285715
-----------------------------------------------------------
Based on product reviews, for  B00GTBZXA6  average rating is  4.225806451612903
The first similar produc

In [39]:
df5_train_target = df3["overall"][:lentrain]
df5_test_target = df3["overall"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

n_neighbors = 3
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)

print(classification_report(df5_test_target, knnpreds_test))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00         2
           3       0.38      0.35      0.36        17
           4       0.80      0.84      0.82        62

    accuracy                           0.72        81
   macro avg       0.39      0.40      0.39        81
weighted avg       0.69      0.72      0.70        81



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
print (accuracy_score(df5_test_target, knnpreds_test))

0.7160493827160493


For 5 neighbors 

In [41]:
df5_train_target = df3["overall"][:lentrain]
df5_test_target = df3["overall"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

n_neighbors = 5
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)
#print (knnpreds_test)

print(classification_report(df5_test_target, knnpreds_test))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00         2
           3       0.54      0.41      0.47        17
           4       0.82      0.90      0.86        62

    accuracy                           0.78        81
   macro avg       0.45      0.44      0.44        81
weighted avg       0.74      0.78      0.76        81



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
print (accuracy_score(df5_test_target, knnpreds_test))

0.7777777777777778
